In [151]:
import pandas  as pd
import numpy as np

from sklearn.feature_selection import VarianceThreshold

In [152]:
relevant_genes = pd.read_csv("DaniDatasets/relevant_genes.csv")

In [153]:
microbiome = pd.read_csv("DaniDatasets/BLCA_csv/microbiome_log_rna.csv",index_col=0)
mirna = pd.read_csv("DaniDatasets/BLCA_csv/miRNA.csv",index_col=0)
svs = pd.read_csv("DaniDatasets/BLCA_csv/svs.csv", index_col=0)
mutations = pd.read_csv("DaniDatasets/BLCA_csv/mutations.csv",index_col=0)
rna_median = pd.read_csv("DaniDatasets/BLCA_csv/RNA_expression_median.csv", index_col=0)
rppa_levels = pd.read_csv("DaniDatasets/BLCA_csv/rppa_levels.csv", index_col=0)
metilation = pd.read_csv("DaniDatasets/BLCA_csv/methylations.csv", index_col=0)
cna_log2 = pd.read_csv("DaniDatasets/BLCA_csv/cna_log2.csv", index_col=0)

In [154]:
for omic in [mutations, metilation, rna_median, cna_log2]:
    columns_to_drop = []
    for column_name in mutations.columns:
        if not (relevant_genes['Gene'] == column_name).any():
            columns_to_drop.append(column_name)
    print("dropped {} columns".format(len(columns_to_drop)))
    omic.drop(columns=columns_to_drop, inplace=True)
    

dropped 0 columns
dropped 0 columns
dropped 0 columns
dropped 0 columns


In [155]:
## svs has a particular configuration
columns_to_drop = []
for column_name in svs.columns:
    gene1, gene2 = column_name.split('_')
    # Check if either gene is in genes_df
    if not ((relevant_genes['Gene'] == gene1).any() 
            or (relevant_genes['Gene'] == gene2).any()):
        columns_to_drop.append(column_name)

# Dropping
svs.drop(columns=columns_to_drop, inplace=True)
svs.shape

(332, 377)

In [156]:
# check for index naming
for i in range(0,len(svs.index)):
    if svs.index[i] != mutations.index[i]:
        print("Concatenation not Correct")
alterations = pd.concat([svs, mutations], axis=1)

In [157]:
# set unique column name
for omic, name in zip([rna_median, metilation, mirna, cna_log2, rppa_levels, alterations, microbiome],
                      ['rna_median', 'metilation', 'mirna', 'cna_log2', 'rppa_levels', 'alterations', 'microbiome']):
    omic.columns = [f"{name}_{c}" for c in omic.columns]

In [158]:
clinical_total = pd.read_csv("DaniDatasets/BLCA_csv/clinical_data.csv", index_col=1)

In [159]:
clinical = clinical_total[['AGE', 'GENDER', 'OS_STATUS', 'OS_MONTHS']]

In [160]:
def convert_status(status):
    if status == '1:DECEASED':
        return 1
    elif status == '0:LIVING':
        return 0
    else:
        print("ERROR: y cannot be empty")
        return np.NaN

def convert_gender(gender):
    if gender == 'Female':
        return 1
    elif gender == 'Male':
        return 0
    else:
        print("ERROR: y cannot be empty in gender")
        return np.NaN


clinical['OS_STATUS'] = clinical['OS_STATUS'].apply(convert_status)
clinical['OS_STATUS'] = pd.to_numeric(clinical['OS_STATUS'])


clinical['GENDER'] = clinical['GENDER'].apply(convert_gender)
clinical['GENDER'] = pd.to_numeric(clinical['GENDER'])

/tmp/ipykernel_2750/2970686832.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clinical['OS_STATUS'] = clinical['OS_STATUS'].apply(convert_status)
/tmp/ipykernel_2750/2970686832.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clinical['OS_STATUS'] = pd.to_numeric(clinical['OS_STATUS'])
/tmp/ipykernel_2750/2970686832.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

In [161]:
# remove patients wit more than 80 yo and less than 6 months OS
dropped = 0
dropped += len(clinical[((clinical['AGE'] >= 80) & 
                         (clinical['OS_MONTHS'] <= 6))])

dropped += len(clinical[clinical['OS_MONTHS']
               .isna() == True])

filtered_clinical = clinical[~(
    (clinical['AGE'] >= 80) & 
    (clinical['OS_MONTHS'] <= 6))]

filtered_clinical = filtered_clinical[filtered_clinical['OS_MONTHS'].isna() == False]

print("dropped {} rows".format(dropped))

dropped 13 rows


In [162]:
print(microbiome.shape, filtered_clinical.shape)

(332, 1406) (319, 4)


In [163]:
d = {}
for k, v in zip(dataframes,
        ["filtered_clinical", "rna_median", "metilation", "mirna", "cna_log2", "rppa_levels", "alterations", "microbiome"]):
    d[v] = k

In [164]:
type(d['filtered_clinical'])

pandas.core.frame.DataFrame

In [169]:
selector = VarianceThreshold() ##default 0.0
dropped_columns = 0

for k in d.keys():
    before = d[k].shape[1]
    d[k] = pd.DataFrame(selector.fit_transform(d[k], ))
    
    after = d[k].shape[1]
    
    dropped = before - after
    print(f"Dropped {dropped} columns in omic: {k}")  
    dropped_columns += dropped
    
print(f"Total dropped columns due to low variance: {dropped_columns}")


Dropped 0 columns in omic: filtered_clinical
Dropped 0 columns in omic: rna_median
Dropped 0 columns in omic: metilation
Dropped 0 columns in omic: mirna
Dropped 0 columns in omic: cna_log2
Dropped 0 columns in omic: rppa_levels
Dropped 0 columns in omic: alterations
Dropped 0 columns in omic: microbiome
Total dropped columns due to low variance: 0


In [170]:
d[k]

,0,1,2,3,4,5,6,7,8,9,...,1396,1397,1398,1399,1400,1401,1402,1403,1404,1405
0,0.733080,0.670418,0.765124,6.730933,0.702300,0.085334,0.082973,5.124396,1.363205,0.659918,...,0.682279,0.850568,0.593939,0.176701,0.744849,0.226570,0.343580,0.607999,1.313179,0.420133
1,0.799478,0.732459,1.099794,6.725545,0.766583,0.562629,0.092474,5.650626,1.463872,0.721209,...,0.745160,0.924610,0.650380,0.196230,0.812043,0.251149,0.379280,0.665494,1.411663,0.462594
2,0.613071,0.558774,0.992511,5.848076,0.586361,0.196826,0.066727,6.171341,2.254735,1.257834,...,0.569027,0.715703,0.492939,0.142989,0.623304,0.183937,0.280987,0.505005,1.129204,0.124470
3,0.707098,0.646193,0.905945,6.557623,0.677172,0.607701,0.079356,6.491229,1.323144,0.635995,...,0.657716,0.821487,0.571960,0.169231,0.718546,0.217146,0.329817,0.585599,1.274030,0.147483
4,0.880366,0.755986,1.021976,7.315561,0.702646,0.468115,0.070132,7.437099,1.236872,0.262879,...,0.767196,0.854524,0.474997,0.200000,0.897650,1.247625,0.205541,0.705577,1.054814,0.358379
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
327,1.872881,0.508706,0.996238,6.874401,0.609905,0.205155,0.084316,6.919992,1.293996,0.273532,...,0.571541,0.704317,0.596845,1.484255,0.664218,0.194443,0.323378,0.499780,1.190346,0.114608
328,0.713288,0.592377,0.707069,6.811078,0.706543,0.385003,0.100680,6.063541,1.456203,5.916935,...,0.663384,1.708202,0.691867,0.844316,0.767405,0.230543,0.380449,0.582257,1.344661,0.136579
329,0.777866,1.939821,0.898217,7.194919,0.770658,0.424125,0.112048,5.317840,1.559987,6.452242,...,0.724482,0.883269,0.754967,0.172444,0.835614,0.255334,0.419172,0.637416,1.443822,0.414511
330,0.669879,1.742324,1.241019,7.071073,0.663457,0.593322,0.093272,4.715758,1.384790,2.004182,...,0.622398,0.764184,0.649488,0.144041,0.721456,0.565916,0.354810,0.545384,1.276619,0.544734


In [123]:
selector = VarianceThreshold() ##default 0.0
dropped_columns = 0

dataframes = [filtered_clinical, rna_median, metilation, mirna, cna_log2, rppa_levels, alterations, microbiome]
transformed_df =[]

for i, (omic, omic_name) in enumerate( 
    zip(dataframes,
        ["filtered_clinical", "rna_median", "metilation", "mirna", "cna_log2", "rppa_levels", "alterations", "microbiome"])):
    before = omic.shape[1] 

    selector.fit_transform()
    pd.DataFrame(transformed_omic, index=omic.index, columns=omic.columns[selector.get_support()])
    
    after = transformed_df[i].shape[1]
    dropped = before - after
    print(f"Dropped {dropped} columns at step {i + 1} : {omic_name}")  
    dropped_columns += dropped

print(f"Total dropped columns due to low variance: {dropped_columns}")


Dropped 0 columns at step 1 : filtered_clinical
Dropped 2 columns at step 2 : rna_median
Dropped 0 columns at step 3 : metilation
Dropped 3 columns at step 4 : mirna
Dropped 0 columns at step 5 : cna_log2
Dropped 0 columns at step 6 : rppa_levels
Dropped 104 columns at step 7 : alterations
Dropped 0 columns at step 8 : microbiome
Total dropped columns due to low variance: 109


In [168]:
type(d[k])

numpy.ndarray

In [166]:
def merge_all(omics):
    df = omics[0]
    starting_indices = [0,omics[0].shape[1]]
    for omic in omics[1:]:
        #print(omic)
        starting_indices.append(starting_indices[-1] + omic.shape[1])
        df = df.merge(omic, left_index=True, right_index=True)
    return (df, starting_indices)

In [103]:
final_df, indices = merge_all(
    [filtered_clinical, 
     rna_median, 
     metilation, 
     mirna ,
     cna_log2, 
     rppa_levels, 
     alterations,
     microbiome])

In [104]:
indices

[0, 4, 2009, 3550, 4293, 6569, 6758, 9165, 10571]

In [17]:
(final_df.shape)

(319, 10571)

In [18]:
# write indices to CSV
pd.DataFrame(indices, columns=['index']).to_csv("DaniDatasets/indices_for_preprocessed.csv", index=False)

In [176]:
final_df = final_df.rename(columns={'OS_STATUS':'event', 'OS_MONTHS': 'time'})

In [177]:
# write df to CSV
# final_df.to_csv("DaniDatasets/preprocessed.csv")

In [178]:
final_df.columns

Index(['AGE', 'GENDER', 'event', 'time', 'rna_median_A1CF', 'rna_median_ABCG2',
       'rna_median_ABI1', 'rna_median_ABI2', 'rna_median_ABL1',
       'rna_median_ABL2',
       ...
       'microbiome_Yokenella', 'microbiome_Yonghaparkia',
       'microbiome_Youngiibacter', 'microbiome_Zavarzinella',
       'microbiome_Zhouia', 'microbiome_Zobellia', 'microbiome_Zooshikella',
       'microbiome_Zunongwangia', 'microbiome_Zymobacter',
       'microbiome_Zymomonas'],
      dtype='object', length=10571)